In [69]:
import pandas as pd
import datetime
import numpy as np

In [70]:
wines_ = pd.read_excel("data/Subset1.xls", 1)
print(min(wines_['DOY']))

213.0


On ne peut donc au maximum prendre comme data que les 213 jours précédent la récolte, une information supplémentaire n'est probablement pas importante car on va surtout analyser l'effet de la météo d'une année sur le vin

In [71]:
wines_.dropna(subset = ['Harvest date'], inplace=True)
wines_.reset_index(drop=True, inplace=True)

In [72]:
weather = pd.read_excel("data/Subset3.xls", [1,2,3,4,5])
weather[1]

,Year,Month,day,Tave,Tmax,Tmin,DOY,Unnamed: 7,Year.1,GST,WI,HI
0,1911,1,1,8.35,16.7,0.0,1,NaN,1911.0,15.073364,1099.050000,967.254052
1,1911,1,2,8.85,19.4,-1.7,2,NaN,1912.0,15.752336,1246.550000,1146.450272
2,1911,1,3,6.65,16.1,-2.8,3,NaN,1913.0,16.722430,1446.300000,1276.779404
3,1911,1,4,8.30,18.3,-1.7,4,NaN,1914.0,15.678579,1217.966001,1103.276910
4,1911,1,5,7.80,18.9,-3.3,5,NaN,1915.0,15.950196,1277.291993,1133.617078
...,...,...,...,...,...,...,...,...,...,...,...,...
36885,2011,12,27,6.95,15.6,-1.7,361,NaN,NaN,NaN,NaN,NaN
36886,2011,12,28,11.65,19.4,3.9,362,NaN,NaN,NaN,NaN,NaN
36887,2011,12,29,11.15,16.7,5.6,363,NaN,NaN,NaN,NaN,NaN
36888,2011,12,30,12.20,15.0,9.4,364,NaN,NaN,NaN,NaN,NaN


In [73]:
i = list(range(1,201))
i = list(map(str, i))
iavg = ['avg' + item for item in i]
imin = ['min' + item for item in i]
imax = ['max' + item for item in i]
temp_avg = pd.DataFrame(columns = iavg)
temp_min = pd.DataFrame(columns = imin)
temp_max = pd.DataFrame(columns = imax)

In [74]:
wines = wines_.copy()

In [75]:
for i in range(len(wines)):
    new_row = []
    r = wines["Reg"][i]
    d = wines['Harvest date'][i]
    h = int(wines['DOY'][i])
    new_row = list(weather[r].loc[(weather[r]['Year'] == d.year) & (weather[r]['DOY'].isin(range(h-200,h)))]['Tave'])
    if len(new_row) != 200:
        new_row = [float('nan')] * 200
    temp_avg.loc[len(temp_avg)] = new_row

In [76]:
for i in range(len(wines)):
    new_row = []
    r = wines["Reg"][i]
    d = wines['Harvest date'][i]
    h = int(wines['DOY'][i])
    new_row = list(weather[r].loc[(weather[r]['Year'] == d.year) & (weather[r]['DOY'].isin(range(h-200,h)))]['Tmin'])
    if len(new_row) != 200:
        new_row = [float('nan')] * 200
    temp_min.loc[len(temp_min)] = new_row

In [77]:
for i in range(len(wines)):
    new_row = []
    r = wines["Reg"][i]
    d = wines['Harvest date'][i]
    h = int(wines['DOY'][i])
    new_row = list(weather[r].loc[(weather[r]['Year'] == d.year) & (weather[r]['DOY'].isin(range(h-200,h)))]['Tmax'])
    if len(new_row) != 200:
        new_row = [float('nan')] * 200
    temp_max.loc[len(temp_max)] = new_row


In [78]:
print(temp_avg.isnull().sum(axis = 0))
print(temp_min.isnull().sum(axis = 0))
print(temp_max.isnull().sum(axis = 0))

avg1      102
avg2      102
avg3      102
avg4      102
avg5      102
         ... 
avg196    102
avg197    102
avg198    102
avg199    102
avg200    102
Length: 200, dtype: int64
min1      102
min2      102
min3      102
min4      102
min5      102
         ... 
min196    102
min197    102
min198    102
min199    102
min200    102
Length: 200, dtype: int64
max1      102
max2      102
max3      102
max4      102
max5      102
         ... 
max196    102
max197    102
max198    102
max199    102
max200    102
Length: 200, dtype: int64


we have the same missing values for Tmin, Tmax and Taverage

In [79]:
temperatures = temp_avg.copy()

In [80]:
# Use pandas get_dummies() function to one-hot encode the 'fruits' column
one_hot_data = pd.get_dummies(wines['Cultivar'])

# Concatenate the one-hot encoded DataFrame with the original DataFrame
dataset2 = pd.concat([wines, one_hot_data], axis=1)

# Drop the original 'fruits' column
dataset2.drop('Cultivar', axis=1, inplace=True)

# Print the resulting DataFrame with one-hot encoded column
dataset2

,Table.No.,Color,Recommend,RecReg,Reg,Harvest date,DOY,Must.Brix,Must.Tacid,Must.pH,...,Trousseau,Valdepes,Verccia Sarda,Verccia bianca,Verdelho,Vermentino Favorita,Walschriesling,White Riesling,Zinfandel,sa Veltliner
0,9,White,Particularly recommended,1.0,3,1936-09-08,251.0,27.2,0.54,3.88,...,0,0,0,0,0,0,0,0,0,0
1,9,White,Particularly recommended,1.0,4,1936-08-31,243.0,23.5,0.67,3.79,...,0,0,0,0,0,0,0,0,0,0
2,9,White,Particularly recommended,1.0,1,1937-10-01,273.0,22.1,0.72,3.38,...,0,0,0,0,0,0,0,0,0,0
3,9,White,Particularly recommended,1.0,3,1937-09-11,253.0,24.6,0.63,3.69,...,0,0,0,0,0,0,0,0,0,0
4,9,White,Particularly recommended,1.0,1,1938-09-14,256.0,19.8,0.78,3.19,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,25,Red,Table grape,NaN,4,1937-10-25,297.0,21.8,0.47,3.74,...,0,0,0,0,0,0,0,0,0,0
1469,25,Red,Table grape,NaN,4,1940-10-07,280.0,22.7,0.36,3.90,...,0,0,0,0,0,0,0,0,0,0
1470,25,Red,Table grape,NaN,4,1935-11-01,304.0,20.2,0.55,NaN,...,0,0,0,0,0,0,0,0,0,0
1471,25,Red,Table grape,NaN,4,1937-10-13,285.0,17.6,0.50,3.58,...,0,0,0,0,0,0,0,0,0,0


### dataset2 ne sera probablement pas utile. En effet on augmente beaucoup trop la dimension par rapport à la quantité de données.

problem : there are a lot of different categories so there are many dimensions added. We already had a dimension to high compared to the amount of observations so this pnly increases the problem.

In [81]:
# Use pandas get_dummies() function to one-hot encode the 'fruits' column
one_hot_data3 = pd.get_dummies(wines['Color'])

# Concatenate the one-hot encoded DataFrame with the original DataFrame
dataset3 = pd.concat([wines, one_hot_data3], axis=1)

# Drop the original 'fruits' column
dataset3.drop('Color', axis=1, inplace=True)

# Print the resulting DataFrame with one-hot encoded column
dataset3

,Table.No.,Cultivar,Recommend,RecReg,Reg,Harvest date,DOY,Must.Brix,Must.Tacid,Must.pH,Wine.Alcohol,Wine.Facid,Wine.Extract,Wine.Tannin,Wine.Tasting,Red,White
0,9,Chardony,Particularly recommended,1.0,3,1936-09-08,251.0,27.2,0.54,3.88,15.6,0.42,4.0,0.03,"Very distinct; good, but alcoholic",0,1
1,9,Chardony,Particularly recommended,1.0,4,1936-08-31,243.0,23.5,0.67,3.79,13.0,0.39,NaN,NaN,Distinct; lacks acid; standard quality,0,1
2,9,Chardony,Particularly recommended,1.0,1,1937-10-01,273.0,22.1,0.72,3.38,12.0,0.52,2.0,0.03,Light body; fruity; distinct; good quality,0,1
3,9,Chardony,Particularly recommended,1.0,3,1937-09-11,253.0,24.6,0.63,3.69,12.8,0.49,2.1,0.03,Flat; tends to oxidize; but distinct,0,1
4,9,Chardony,Particularly recommended,1.0,1,1938-09-14,256.0,19.8,0.78,3.19,10.5,0.68,2.8,0.07,"Tart, but palatable; distinct; picked too early",0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,25,Red Malaga (Molinera),Table grape,NaN,4,1937-10-25,297.0,21.8,0.47,3.74,12.0,0.28,2.3,0.04,Very flat; some spoilage; very ordiry sherry,1,0
1469,25,Red Malaga (Molinera),Table grape,NaN,4,1940-10-07,280.0,22.7,0.36,3.90,12.4,0.40,2.8,0.04,Flat; pink; very common; lacks everything,1,0
1470,25,Ribier,Table grape,NaN,4,1935-11-01,304.0,20.2,0.55,NaN,10.8,0.34,2.5,0.08,"Flat; poor; sherry; soft, but no character",1,0
1471,25,Ribier,Table grape,NaN,4,1937-10-13,285.0,17.6,0.50,3.58,9.5,0.35,2.1,0.06,Flat; ordiry; tends to spoil,1,0


### On a seulement deux catégories donc la notion d'équidistence entre chaque catégorie n'est pas forcément importante, on peut se contenter d'encoder par 0 1 ou -1 1 et par la même occasion ne pas augmenter non nécessairement la dimension.

In [82]:
mapping = {'White': -1, 'Red': 1}

# Apply the mapping to the 'category' column using the map() function
wines['Color'] = wines['Color'].map(mapping)

wines

,Table.No.,Cultivar,Color,Recommend,RecReg,Reg,Harvest date,DOY,Must.Brix,Must.Tacid,Must.pH,Wine.Alcohol,Wine.Facid,Wine.Extract,Wine.Tannin,Wine.Tasting
0,9,Chardony,-1,Particularly recommended,1.0,3,1936-09-08,251.0,27.2,0.54,3.88,15.6,0.42,4.0,0.03,"Very distinct; good, but alcoholic"
1,9,Chardony,-1,Particularly recommended,1.0,4,1936-08-31,243.0,23.5,0.67,3.79,13.0,0.39,NaN,NaN,Distinct; lacks acid; standard quality
2,9,Chardony,-1,Particularly recommended,1.0,1,1937-10-01,273.0,22.1,0.72,3.38,12.0,0.52,2.0,0.03,Light body; fruity; distinct; good quality
3,9,Chardony,-1,Particularly recommended,1.0,3,1937-09-11,253.0,24.6,0.63,3.69,12.8,0.49,2.1,0.03,Flat; tends to oxidize; but distinct
4,9,Chardony,-1,Particularly recommended,1.0,1,1938-09-14,256.0,19.8,0.78,3.19,10.5,0.68,2.8,0.07,"Tart, but palatable; distinct; picked too early"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,25,Red Malaga (Molinera),1,Table grape,NaN,4,1937-10-25,297.0,21.8,0.47,3.74,12.0,0.28,2.3,0.04,Very flat; some spoilage; very ordiry sherry
1469,25,Red Malaga (Molinera),1,Table grape,NaN,4,1940-10-07,280.0,22.7,0.36,3.90,12.4,0.40,2.8,0.04,Flat; pink; very common; lacks everything
1470,25,Ribier,1,Table grape,NaN,4,1935-11-01,304.0,20.2,0.55,NaN,10.8,0.34,2.5,0.08,"Flat; poor; sherry; soft, but no character"
1471,25,Ribier,1,Table grape,NaN,4,1937-10-13,285.0,17.6,0.50,3.58,9.5,0.35,2.1,0.06,Flat; ordiry; tends to spoil


In [83]:
from sklearn.feature_extraction import FeatureHasher


# Instantiate a FeatureHasher object with the desired number of output features (n_features)
hasher = FeatureHasher(n_features=20, input_type='string')
cultivar_list = [[x] for x in wines['Cultivar'].tolist()]
# Perform feature hashing on the 'fruits' column
hashed_features = hasher.fit_transform(cultivar_list)

# Convert the hashed features to a dense NumPy array and create a DataFrame
hashed_features_df = pd.DataFrame(hashed_features.toarray())

# Concatenate the hashed features DataFrame with the original DataFrame
dataset4 = pd.concat([wines, hashed_features_df], axis=1)

# Drop the original 'fruits' column
dataset4.drop('Cultivar', axis=1, inplace=True)
dataset4.head()


,Table.No.,Color,Recommend,RecReg,Reg,Harvest date,DOY,Must.Brix,Must.Tacid,Must.pH,...,10,11,12,13,14,15,16,17,18,19
0,9,-1,Particularly recommended,1.0,3,1936-09-08,251.0,27.2,0.54,3.88,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
1,9,-1,Particularly recommended,1.0,4,1936-08-31,243.0,23.5,0.67,3.79,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
2,9,-1,Particularly recommended,1.0,1,1937-10-01,273.0,22.1,0.72,3.38,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
3,9,-1,Particularly recommended,1.0,3,1937-09-11,253.0,24.6,0.63,3.69,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
4,9,-1,Particularly recommended,1.0,1,1938-09-14,256.0,19.8,0.78,3.19,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0


In [84]:
dataset = pd.concat([wines,temp_avg], axis = 1)

### On utilisera cet encodage par la suite dans certaines analyses. la méthode par arbre nous permettra quand à elle de rester sur des données encodées par des str

In [85]:
dataset.nunique(axis=0)

Table.No.     17
Cultivar     148
Color          2
Recommend      5
RecReg         5
            ... 
avg196       189
avg197       185
avg198       185
avg199       185
avg200       190
Length: 216, dtype: int64

In [86]:
print(dataset.isna().sum()) #il faudrait trouver et enlever les 102 wines ou on a pas la temperature selon mon avis 

Table.No.       0
Cultivar        0
Color           0
Recommend       0
RecReg       1025
             ... 
avg196        102
avg197        102
avg198        102
avg199        102
avg200        102
Length: 216, dtype: int64


# Mise en place d'une analyse avec tout d'abord la color du wine comme attribute et le ph à prédire

In [87]:
copie = dataset.copy()
#enlever les colonnes qu'on ne compte pas utiliser
copie = copie.drop(columns = ['Table.No.', 'Cultivar', 'Recommend', 'RecReg', 'Reg', 'Wine.Alcohol',
       'Harvest date', 'DOY', 'Must.Brix', 'Must.Tacid', 'Wine.Facid', 'Wine.Extract', 'Wine.Tannin',
       'Wine.Tasting'])
#verifier qu'on a nos colonnes désirées
copie.columns 

Index(['Color', 'Must.pH', 'avg1', 'avg2', 'avg3', 'avg4', 'avg5', 'avg6',
       'avg7', 'avg8',
       ...
       'avg191', 'avg192', 'avg193', 'avg194', 'avg195', 'avg196', 'avg197',
       'avg198', 'avg199', 'avg200'],
      dtype='object', length=202)

In [88]:
columns = list(temperatures.columns)
columns.append('Color')
columns.append('Must.pH')
#delete row with value = Nan
#voir si pour les y on peut pas remplacer par une valeur :
# d'autres possibilités (ffill et bfill qui propagent les valeurs observées aux voisines Nan)
# imputation par la moyenne (possible aussi avec la mediane)
# exemple :           movies_df['budget'].fillna(movies_df['budget'].mean(), inplace=True)
copie.dropna(subset = columns, inplace=True)
copie.reset_index(drop=True, inplace=True)

In [89]:
# selection of class to predict : here ph
y = copie[['Must.pH']]

# temperature and color extraction
colonnes = list(temperatures.columns)
colonnes.append('Color')

# récupération des valeurs de toutes les colonnes à la suite
X = copie[colonnes]

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [91]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
#on fit (calcule mean et std) sur toutes les colonnes temperature a la fois pas que les colonnes
scaler.fit(X_train[list(temperatures.columns)].values.ravel().reshape(-1, 1))

#on remplace nos colonnes par nos valeurs standardizer
for col in temperatures.columns:
    X_train[col] = scaler.transform(X_train[[col]])
    X_test[col] = scaler.transform(X_test[[col]])

X_train

/home/augustin/Documents/M1/projectML/project/projet_wine/.conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/augustin/Documents/M1/projectML/project/projet_wine/.conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/augustin/Documents/M1/projectML/project/projet_wine/.conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/augustin/Documents/M1/projectML/project/projet_wine/.conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/augustin/Documents/M1/projectML/project/projet_wine/.conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: 

,avg1,avg2,avg3,avg4,avg5,avg6,avg7,avg8,avg9,avg10,...,avg192,avg193,avg194,avg195,avg196,avg197,avg198,avg199,avg200,Color
296,-2.368635,-1.745267,-1.758056,-1.914169,-2.085856,-1.863673,-1.474853,-1.722284,-1.444555,-1.187024,...,-0.884047,-0.909295,-0.990089,-0.434631,-1.075933,-1.217322,-0.742658,-0.717410,0.242018,-1
599,-1.020387,-0.747707,-1.080982,-0.576020,-1.020387,-1.030486,-1.363761,-1.636440,-1.414257,-1.757631,...,-0.010463,-0.687112,-0.687112,-0.293242,0.544995,-0.576020,-0.071059,-0.636616,-0.454830,1
711,-1.414257,-1.868723,-1.919219,-1.303166,-1.919219,-2.030311,-2.030311,-1.464753,-1.808127,-2.201998,...,2.170972,1.777102,1.332735,1.282239,1.342835,1.565018,1.565018,1.787201,2.059881,1
274,-1.242570,-1.424356,-0.959791,-0.909295,-1.252669,-1.535448,-1.020387,-1.303166,-1.525349,-1.697036,...,-0.515425,-0.182150,0.322812,0.161224,0.050132,0.161224,-0.010463,-0.182150,-0.293242,-1
362,-1.303166,-1.424356,-1.141578,-1.192074,-0.454830,-1.030486,-1.525349,-1.363761,-1.585944,-1.020387,...,2.453751,1.837697,2.231568,2.231568,1.332735,1.110552,1.332735,1.161048,1.221644,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,-2.030311,-1.131478,-2.030311,-2.030311,-2.040410,-1.585944,-1.919219,-1.858624,-2.201998,-2.424181,...,0.373308,0.211720,0.433903,0.272316,0.494499,1.443827,0.767178,1.272140,1.272140,1
1095,-0.798204,-0.131654,-1.020387,-0.677013,-0.798204,-0.242746,-1.020387,-1.414257,-1.020387,-1.575845,...,1.837697,1.221644,-0.798204,0.100628,0.555094,-0.071059,-0.010463,-0.020562,-0.071059,-1
1130,-1.242570,-0.353837,-0.576020,-0.969891,-0.919394,-1.303166,-1.646540,-1.192074,-1.141578,-0.798204,...,-0.464929,-0.909295,-0.909295,-1.242570,-0.909295,-1.030486,-1.414257,-0.858799,-0.909295,1
860,-2.030311,-1.303166,-0.969891,-0.737608,-0.677013,-0.737608,-0.747707,-0.737608,-0.454830,-0.737608,...,-0.242746,0.544995,0.716682,0.827774,0.433903,1.110552,1.453926,1.615514,1.615514,-1


### a definir si on veut faire avec adaline perceptron svm ....
### code exemple mais pas utilisé pour l'instant, on fera peutêtre un cross valisation entre les lasso, cv , ... 

```python
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, accuracy_score

# Définition des paramètres de la K-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # objet KFold

# Initialisation de la liste des erreurs pour chaque fold
fold_errors = []

# Boucle sur chaque fold de la K-fold cross-validation
for train_index, test_index in kf.split(X):
    # Sélection des données d'apprentissage et de test pour ce fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Entraînement du modèle sur les données d'apprentissage
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Évaluation du modèle sur les données de test
    y_pred = model.predict(X_test)
    fold_error = mean_squared_error(y_test, y_pred)
    fold_errors.append(fold_error)

# Calcul de la moyenne et de l'écart-type des erreurs pour tous les folds
mean_error = sum(fold_errors) / len(fold_errors)
std_error = np.std(fold_errors)

# Affichage des résultats
print(f"K-fold cross-validation ({10} folds) :")
print(f"  Mean squared error : {mean_error:.2f}")
print(f"  Standard deviation : {std_error:.2f}")
```

# Analyse sur la donnée de tous les jours avec regularization. 
## on fait 
* Linéaire (pas vraiment)
* Ridge Regression
* Lasso Regression
* Support Vector Regression
* Random Forest Regression/ Decision Tree Regression
* kernelized regression

## reste à faire 
* Analyser avec plus de données comme Tmin, Tmax, un mix

## À réctifier: 
* standardization sur seulement l'ensemble train,
* Implémenter les métriques (accuracy, recall, etc)

In [92]:
import numpy as np
from sklearn.linear_model import LassoCV, RidgeCV

#linear regression avec lasso et ridge 
alphas = np.logspace(-5, 2, num=100)
alphas2 = 1/alphas
lasso_cv = LassoCV(alphas=alphas, cv=5)
ridge_cv = RidgeCV(alphas=alphas2, cv=5)
lasso_cv.fit(X_train, y_train.values.ravel())
ridge_cv.fit(X_train, y_train.values.ravel())
print("Lasso Best alpha:", lasso_cv.alpha_)
print("Ridge Best alpha:", 1/ridge_cv.alpha_)



Lasso Best alpha: 0.002154434690031882
Ridge Best alpha: 0.010974987654930556


In [93]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

In [94]:
c_values = np.logspace(-3, .2, num=3)


In [95]:
param_grid_lin = {'C': c_values, 
              'epsilon': [0.01, 0.1, 1]}
grid_lin = GridSearchCV(SVR(kernel='linear'), param_grid=param_grid_lin, cv=5)

grid_lin.fit(X_train, y_train.values.ravel())

print("Best hyperparameters: ", grid_lin.best_params_)
print("Cross-validation score: ", grid_lin.best_score_)

Best hyperparameters:  {'C': 0.001, 'epsilon': 0.1}
Cross-validation score:  0.2517434390994759


In [96]:
param_grid_rbf = {'C': c_values, 
              'epsilon': [0.01, 0.1, 1],
              'gamma': ['scale', 'auto']}
grid_rbf = GridSearchCV(SVR(kernel='rbf'), param_grid=param_grid_rbf, cv=5)

grid_rbf.fit(X_train, y_train.values.ravel())

print("Best hyperparameters: ", grid_rbf.best_params_)
print("Cross-validation score: ", grid_rbf.best_score_)

Best hyperparameters:  {'C': 1.5848931924611136, 'epsilon': 0.1, 'gamma': 'scale'}
Cross-validation score:  0.39902530301112876


In [97]:
param_grid_poly = {'C': c_values, 
              'epsilon': [0.01, 0.1, 1],
              'degree':[2,3,4,5,6,7]}
grid_poly = GridSearchCV(SVR(kernel='poly'), param_grid=param_grid_poly, cv=5)

grid_poly.fit(X_train, y_train.values.ravel())

print("Best hyperparameters: ", grid_poly.best_params_)
print("Cross-validation score: ", grid_poly.best_score_)
print("Cross-validation results: ", grid_poly.cv_results_)
print(grid_poly.scorer_)

Best hyperparameters:  {'C': 1.5848931924611136, 'degree': 2, 'epsilon': 0.1}
Cross-validation score:  0.378283937890696
Cross-validation results:  {'mean_fit_time': array([0.04757433, 0.03510623, 0.00375953, 0.0481667 , 0.0366312 ,
       0.00378132, 0.05289497, 0.03545637, 0.0041327 , 0.04980793,
       0.03550076, 0.00386443, 0.04913707, 0.03766818, 0.00402384,
       0.04931035, 0.03585668, 0.00378585, 0.04659405, 0.03414183,
       0.00379553, 0.04719653, 0.03525567, 0.00375509, 0.04785938,
       0.03539357, 0.00373135, 0.04895926, 0.03686576, 0.00407972,
       0.05028577, 0.03666167, 0.00374393, 0.05002046, 0.03758287,
       0.00369191, 0.06330466, 0.04389424, 0.00372944, 0.06045446,
       0.04393921, 0.00366602, 0.05986032, 0.04307113, 0.00365849,
       0.05937848, 0.04212828, 0.00385485, 0.0591557 , 0.0445416 ,
       0.00386243, 0.05962729, 0.04402785, 0.00374689]), 'std_fit_time': array([1.27972028e-03, 4.83678365e-04, 3.03973145e-04, 2.87739938e-03,
       1.74066135e-0

In [98]:
param_grid_sig = {'C': c_values, 
              'epsilon': [0.01, 0.1, 1]}
grid_sig = GridSearchCV(SVR(kernel='sigmoid'), param_grid=param_grid_sig, cv=5)

grid_sig.fit(X_train, y_train.values.ravel())

print("Best hyperparameters: ", grid_sig.best_params_)
print("Cross-validation score: ", grid_sig.best_score_)

Best hyperparameters:  {'C': 0.039810717055349734, 'epsilon': 0.1}
Cross-validation score:  0.12050367751341962


In [99]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np


# Define the hyperparameter grid for GridSearchCV
param_grid_rt = {
    'n_estimators': [10, 50, 100],
    'max_depth': [2, 4, 6, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Create a KFold cross-validation object
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# Create a Random Forest model
rf = RandomForestRegressor()

# Create a GridSearchCV object
grid_rt = GridSearchCV(estimator=rf, param_grid=param_grid_rt, cv=5, scoring='neg_mean_squared_error', n_jobs=2)

# Fit the GridSearchCV object to the data
grid_rt.fit(X_train, y_train.values.ravel())

# Print the best hyperparameters and the corresponding mean cross-validation score
print("Best hyperparameters:", grid_rt.best_params_)
print("Best cross-validation score:", -grid_rt.best_score_)


Best hyperparameters: {'max_depth': 6, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}
Best cross-validation score: 0.04528237925204606


In [100]:
from scipy.stats import ttest_rel
from scipy.stats import shapiro


model1 = grid_poly
model2 = grid_lin

# Extract the cross-validation scores for each model
scores1 = model1.cv_results_['mean_test_score']
scores2 = model2.cv_results_['mean_test_score']

# Check for normality using the Shapiro-Wilk test
_, p1 = shapiro(scores1)
_, p2 = shapiro(scores2)

# Perform a paired t-test if the scores are normally distributed
if p1 > 0.05 and p2 > 0.05:
    _, p = ttest_rel(scores1, scores2)
    print('Paired t-test p-value:', p)
else:
    print('The scores are not normally distributed')

The scores are not normally distributed


In [101]:
len(scores1)

54

In [102]:
from scipy.stats import wilcoxon

# Perform a Wilcoxon signed-rank test if the scores are not normally distributed
w, p = wilcoxon(scores1, scores2)
print("Wilcoxon signed-rank test:")
print("Statistic:", w)
print("p-value:", p)

ValueError: The samples x and y must have the same length.